In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ket-2k/Ket.xlsx
/kaggle/input/final-10k/TenK.xlsx


In [2]:
# Install required libraries
!pip install transformers
!pip install datasets
!pip install torch
!pip install openpyxl  # Required for reading Excel files
!pip install tqdm  # Progress bar
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ff392b1f29dd1fe0d5244966441c8e850ce737d8f7e37f695065f80d82001912
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
import os
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import evaluate
import torch
import gc
from tqdm import tqdm  # Import tqdm for progress bar



In [4]:
import transformers
print("Transformers version:", transformers.__version__)

Transformers version: 4.47.0


In [5]:
# Disable wandb logging
os.environ["WANDB_DISABLED"] = "true"

# Step 1: Load the dataset from Kaggle input directory
df = pd.read_excel("/kaggle/input/final-10k/TenK.xlsx")  # Adjust path to Kaggle input directory

# Ensure the dataset has two columns: 'Text' and 'Summary'
df = df.rename(columns={df.columns[0]: "Text", df.columns[1]: "Summary"})
df = df.dropna()

# Optionally, limit the dataset size for testing 
#df = df.head(100)


In [6]:


# Step 2: Split the dataset into train, validation, and test sets (70% train, 20% val, 20% test)
train_df, temp_df = train_test_split(df, test_size=0.20, random_state=42)  # 70% for training, 30% remaining
val_df, test_df = train_test_split(temp_df, test_size=0.50, random_state=42)  # Split the remaining 30% into 50% val and 50% test

# Convert pandas DataFrames into Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Combine datasets into a DatasetDict for Hugging Face Trainer
dataset_dict = DatasetDict({"train": train_dataset, "val": val_dataset, "test": test_dataset})


In [7]:

# Step 3: Load the tokenizer and model (using MT5 for summarization)
model_name = "google/mt5-small"  # MT5 model for summarization

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Clear RAM before loading a new model or changing configurations (optional step for memory management)
def clear_ram():
    """Clear up the GPU memory and release model from RAM."""
    print("Clearing RAM...")
    torch.cuda.empty_cache()  # Clear GPU cache
    gc.collect()  # Run garbage collection to release other resources
    print("RAM cleared.")

clear_ram()

# Load tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)  # Use fast tokenizer for better performance
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Move model to the selected device (GPU or CPU)
model.to(device)


Using device: cuda
Clearing RAM...
RAM cleared.


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [8]:
# Tokenization function for summarization
def preprocess_function(examples):
    # Prepend <extra_id_0> to the input text for MT5 (for summarization tasks)
    inputs = ["<extra_id_0> " + text for text in examples['Text']]

    # Tokenize the input text (max_length and truncation ensure input fits the model's constraints)
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)

    # Tokenize the summary (summary is used as labels)
    labels = tokenizer(examples['Summary'], truncation=True, padding="max_length", max_length=512)

    # Add the labels to the inputs dictionary
    model_inputs['labels'] = labels['input_ids']

    # Mask padding tokens in the labels by setting them to -100 (this tells the model to ignore the padding tokens during loss calculation)
    model_inputs["labels"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_ids]
        for label_ids in model_inputs["labels"]
    ]

    return model_inputs

# Apply tokenization to the train, validation, and test datasets using the `map` function
tokenized_datasets = dataset_dict.map(preprocess_function, batched=True)


Map:   0%|          | 0/8006 [00:00<?, ? examples/s]

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

In [9]:


# Step 4: Set up training arguments (adjust batch size here)
training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/results",  # Save results to Kaggle's working directory
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=3,  # Adjusted batch size to 4
    per_device_eval_batch_size=3,   # Adjusted batch size to 4
    num_train_epochs=22,
    weight_decay=0.01,
    logging_dir='/kaggle/working/logs',  # Save logs to Kaggle's working directory
    logging_steps=10,
    predict_with_generate=True,
    run_name="mt5_summarization_run",  # Optional: set a different run name
    report_to="none",  # Disable logging to external platforms like WandB
    #save_total_limit=2,  #-----------1
    #save_strategy="epoch",
    save_strategy='no',
    fp16=True,  # Mixed precision for memory efficiency
    #gradient_accumulation_steps=2, #increase effective batch size, requires less epoch kintu slow

)


In [10]:


# Step 5: Initialize and train with the custom trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
 
)




<ipython-input-10-f2409f71d40e>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# Step 7: Train the model
trainer.train()


# Step 8: Save the fine-tuned model and tokenizer to Kaggle's working directory
def save_model(model, output_dir):
    for param in model.parameters():
        if not param.is_contiguous():
            param.data = param.data.contiguous()
    model.save_pretrained(output_dir)

# Saving model and tokenizer in Kaggle's working directory
save_model(model, "/kaggle/working/fine_tuned_mt5")
tokenizer.save_pretrained("/kaggle/working/fine_tuned_mt5")


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.948800,2.271500
2,2.593900,2.117457


In [ ]:


# Step 9: Clear GPU memory after training to save resources
clear_ram()

# Step 10: Load the model for inference (summary generation)
def load_model_for_inference(model_path, device):
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model.to(device)  # Move model to the selected device
    return model, tokenizer

# Step 11: Load the model and tokenizer for inference
model_path = "/kaggle/working/fine_tuned_mt5"  # Changed to Kaggle's working directory
model, tokenizer = load_model_for_inference(model_path, device)

# Step 12: Generate summary using the fine-tuned model
def generate_summary(text, model, tokenizer, device):
    model.to(device)

    # Remove <extra_id_0> from the input text (for inference)
    text = text.replace("<extra_id_0>", "").strip()

    # Tokenize the input text
    inputs = tokenizer(text, max_length=512, truncation=True, return_tensors="pt", padding="max_length")

    # Move input tensors to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate summary
    summary_ids = model.generate(
        inputs['input_ids'],
        max_length=1024,
        num_beams=4,  #3
        no_repeat_ngram_size=4, #2
        early_stopping=True
    )

    # Decode and return the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Step 13: Generate summaries for the test dataset using tqdm progress bar
tqdm.pandas(desc="Generating summaries")  # Set the description for tqdm
test_df['generated_summary'] = test_df['Text'].progress_apply(lambda x: generate_summary(x, model, tokenizer, device))

# Step 14: Load the ROUGE metric from the 'evaluate' library
rouge = evaluate.load('rouge')

# Step 15: Calculate the final ROUGE score for all predictions and references
references = test_df['Summary'].tolist()
predictions = test_df['generated_summary'].tolist()

# Compute the final ROUGE score across the whole validation set
final_rouge_score = rouge.compute(predictions=predictions, references=references)

# Print the final ROUGE scores
print(f"Final ROUGE Score: {final_rouge_score}")

# Save the output to a new Excel file with generated summaries and final ROUGE scores
test_df.to_excel("/kaggle/working/test_data_with_summaries_mt5.xlsx", index=False)  # Save in Kaggle working directory

# Step 16: Save the second Excel file by removing the <extra_id_0> token from the generated summaries
test_df['generated_summary_no_extra'] = test_df['generated_summary'].apply(lambda x: x.replace("<extra_id_0>", "").strip())

# Save the second Excel file
test_df.to_excel("/kaggle/working/test_data_with_summaries_no_extra_mt5.xlsx", index=False)  # Save in Kaggle working directory

# Save the second file with only the 'generated_summary_no_extra' column
#test_df[['Text', 'Summary', 'generated_summary_no_extra']].to_excel("/kaggle/working/test_data_with_summaries_no_extra_mt5.xlsx", index=False)

# Print the result with generated summaries
print(test_df[['Text', 'Summary', 'generated_summary', 'generated_summary_no_extra']])
